## Import

In [38]:
import numpy as np
from abc import ABC, abstractmethod
from typing import List, Tuple
import tensorflow as tf
import keras

## Preprocessing

In [42]:
(X_train_raw, Y_train), (X_test_raw, Y_test) = tf.keras.datasets.mnist.load_data()

In [44]:
X_train = X_train_raw.astype("float32")/255.0
X_test = X_test_raw.astype("float32")/255.0

X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

Y_train = tf.keras.utils.to_categorical(Y_train, 10)
Y_test = tf.keras.utils.to_categorical(Y_test, 10)

## From scratch

In [8]:
class Module(ABC) :

    @abstractmethod
    def forward(self, x) : pass
    
    @abstractmethod
    def backward(self, delta_out) : pass

In [19]:
class SimpleLayer(Module) :

    @staticmethod
    def init_weights_bias(input_size : int, output_size : int) -> Tuple[np.ndarray, np.ndarray] :
        weights = np.random.uniform(-10,10, (input_size,output_size))
        bias = np.zeros(output_size)
        return weights, bias
    
    def __init__ (self, input_size : int, output_size : int) -> None :
        self.weights, self.bias = SimpleLayer.init_weights_bias(input_size, output_size)

    def forward(self, x : np.ndarray) -> np.ndarray:
        self.input = x
        return x@self.weights+self.bias
    
    def update(self, lr : float=10e-5) -> None :
        assert hasattr(self, "input")
        self.weights -= lr*self.input@self.delta
        self.bias -= lr*np.sum(self.delta)

    
    def backward(self, delta_out : np.ndarray) :
        self.delta = delta_out@self.weights.T
        self.update()


In [13]:
class Sequential(Module) :
    def __init__(self, layers=None) :
        self.layers = layers

    def forward(self, x) :
        for layer in self.layers :
            x = layer.forward(x)
        return x

    def backward(self, delta_out) :
        for layer in reversed(self.layers) :
            delta_out = layer.backward(delta_out)
        return delta_out


In [32]:
class CELoss(Module) :
    def __init__(self, size=2) :
        self.size = size
    
    def forward(self, x) : pass
    def backward(self, delta_out) : pass

In [37]:
class Identity(Module) : pass

In [ ]:
class Sigmoid(Module) :
    

In [35]:
class DropoutLayer(Module) : pass

In [36]:
class BatchNorm(Module) : pass

In [34]:
class EarlyStopper(Module) : pass

In [30]:
class Model() :
    BATCH_SIZE = 4
    NB_EPOCHS = 1
    def __init__(self, sequential, loss) :
        self.sequential = sequential
        self.loss = loss
        self.metrics = {}

    def train(self, X, Y) :
        self.metrics["train_loss"] = []

        for epoch in Model.NB_EPOCHS :
            loss_value = 0
            for k in range(0,len(X), Model.BATCH_SIZE) :
                batch = X[k:k+Model.BATCH_SIZE]
                batch_label = Y[k:k+Model.BATCH_SIZE]
                Y_hat = self.sequential.forward(batch)
                loss_value += self.loss.forward(Y, Y_hat)

                delta_out = self.loss.backward(Y, Y_hat)
                self.sequential.backward(delta_out)

            self.metrics["train_loss"].append(loss_value)

    def test(self, X, Y) :
        self.metrics["test_loss"] = []
        loss_value = 0
        for k in range(0,len(X), Model.BATCH_SIZE) :
            batch = X[k:k+Model.BATCH_SIZE]
            batch_label = Y[k:k+Model.BATCH_SIZE]
            Y_hat = self.sequential.forward(batch)
            loss_value += self.loss.forward(Y, Y_hat)
        self.metrics["test_loss"].append(loss_value)

In [33]:
layer1 = SimpleLayer(3,3)
layer2 = SimpleLayer(3,4)
loss = CELoss()
model = Model(Sequential([layer1, layer2]), loss)

## Keras

In [47]:
model = keras.models.Sequential( [
                                 keras.layers.Dense(128, "relu", input_shape=(28*28,)),
                                 keras.layers.Dropout(0.2),
                                 keras.layers.Dense(10, "softmax")]
                                 )
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, Y_train, batch_size=16, epochs=5)

Epoch 1/5
3750/3750 [==============================] - 11s 3ms/step - loss: 0.2707 - accuracy: 0.9210
Epoch 2/5
3750/3750 [==============================] - 10s 3ms/step - loss: 0.1355 - accuracy: 0.9596
Epoch 3/5
3750/3750 [==============================] - 10s 3ms/step - loss: 0.1033 - accuracy: 0.9687
Epoch 4/5
3750/3750 [==============================] - 10s 3ms/step - loss: 0.0857 - accuracy: 0.9733
Epoch 5/5
3750/3750 [==============================] - 9s 3ms/step - loss: 0.0747 - accuracy: 0.9764


In [48]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0759 - accuracy: 0.9765


[0.07587943226099014, 0.9764999747276306]

## PyTorch